In [3]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_161"; Java(TM) SE Runtime Environment (build 1.8.0_161-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.161-b12, mixed mode)
  Starting server from /Users/hetty/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/7l/m1x60k3109j9nxx82v_nykc80000gn/T/tmpix6mk0ds
  JVM stdout: /var/folders/7l/m1x60k3109j9nxx82v_nykc80000gn/T/tmpix6mk0ds/h2o_hetty_started_from_python.out
  JVM stderr: /var/folders/7l/m1x60k3109j9nxx82v_nykc80000gn/T/tmpix6mk0ds/h2o_hetty_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,08 secs
H2O cluster version:,3.16.0.4
H2O cluster version age:,"21 days, 19 hours and 52 minutes"
H2O cluster name:,H2O_from_python_hetty_iiic0f
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [5]:
data = {
    'tra': pd.read_csv('air_visit_data.csv'),
    'as': pd.read_csv('air_store_info.csv'),
    'hs': pd.read_csv('hpg_store_info.csv'),
    'ar': pd.read_csv('air_reserve.csv'),
    'hr': pd.read_csv('hpg_reserve.csv'),
    'id': pd.read_csv('store_id_relation.csv'),
    'tes': pd.read_csv('sample_submission.csv'),
    'hol': pd.read_csv('date_info.csv').rename(columns={'calendar_date':'visit_date'}),
    'before': pd.read_csv('before_42D_data.csv')
    }

In [6]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])
data['hr'] = pd.merge(data['hr'], data['as'], how='left', on=['air_store_id'])
data['ar'] = pd.merge(data['ar'], data['as'], how='left', on=['air_store_id'])

In [7]:
tmp_area = {}
tmp_genre = {}

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['v_r_diff'] = data[df].apply(
        lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp_area[df] = data[df].groupby(
        ['air_area_name','visit_datetime'],as_index=False)['reserve_visitors'].median().rename(
        columns={'visit_datetime':'visit_date','reserve_visitors':'area_reserve_visitors'})
    tmp_genre[df] = data[df].groupby(
        ['air_genre_name','visit_datetime'],as_index=False)['reserve_visitors'].median().rename(
        columns={'visit_datetime':'visit_date','reserve_visitors':'genre_reserve_visitors'})
    data[df] = data[df].groupby(
        ['air_store_id','visit_datetime'], 
        as_index=False)['reserve_visitors'].sum().rename(columns={'visit_datetime':'visit_date'})    

In [9]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['week'] = data['tra']['visit_date'].dt.week
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [10]:
data['tes']['visit_date'] = data['tes']['id'].map(
    lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(
    lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['week'] = data['tes']['visit_date'].dt.week
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [12]:
tmp = data['tra'].groupby(
    ['air_store_id','dow'],as_index=False)['visitors'].min().rename(columns={
    'visitors':'min_visitors'})
data['tra'] = pd.merge(data['tra'], tmp, how='left', on=['air_store_id','dow'])
data['tes'] = pd.merge(data['tes'], tmp, how='left', on=['air_store_id','dow'])

In [13]:
tmp = data['tra'].groupby(
    ['air_store_id','dow'],as_index=False)['visitors'].max().rename(columns={
    'visitors':'max_visitors'})
data['tra'] = pd.merge(data['tra'], tmp, how='left', on=['air_store_id','dow'])
data['tes'] = pd.merge(data['tes'], tmp, how='left', on=['air_store_id','dow'])

In [14]:
tmp = data['tra'].groupby(
    ['air_store_id','dow'],as_index=False)['visitors'].mean().rename(columns={
    'visitors':'mean_visitors'})
data['tra'] = pd.merge(data['tra'], tmp, how='left', on=['air_store_id','dow'])
data['tes'] = pd.merge(data['tes'], tmp, how='left', on=['air_store_id','dow'])

In [15]:
tmp = data['tra'].groupby(
    ['air_store_id','dow'],as_index=False)['visitors'].median().rename(columns={
    'visitors':'median_visitors'})
data['tra'] = pd.merge(data['tra'], tmp, how='left', on=['air_store_id','dow'])
data['tes'] = pd.merge(data['tes'], tmp, how='left', on=['air_store_id','dow'])

In [16]:
tmp = data['tra'].groupby(
    ['air_store_id','dow'],as_index=False)['visitors'].count().rename(columns={
    'visitors':'count_visitors'})
data['tra'] = pd.merge(data['tra'], tmp, how='left', on=['air_store_id','dow'])
data['tes'] = pd.merge(data['tes'], tmp, how='left', on=['air_store_id','dow'])

In [17]:
data['before']['visit_date'] = pd.to_datetime(data['before']['visit_date'])
data['before']['visit_date'] = data['before']['visit_date'].dt.date
data['tra'] = pd.merge(data['tra'], data['before'], how='left', on=['air_store_id','visit_date'])
data['tes'] = pd.merge(data['tes'], data['before'], how='left', on=['air_store_id','visit_date'])

In [18]:
data['tra'] = pd.merge(data['tra'], data['as'], how='left', on=['air_store_id'])
data['tes'] = pd.merge(data['tes'], data['as'], how='left', on=['air_store_id'])

In [19]:
del data['hol']['day_of_week']

In [20]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

In [21]:
data['tra'] = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date'])
data['tes'] = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date'])

In [22]:
train_r = pd.merge(data['ar'], data['hr'], 
                   how='outer', on=['air_store_id','visit_date']).fillna(0)
train_r['reserve_visitor'] = train_r['reserve_visitors_x'] + train_r['reserve_visitors_y']

In [23]:
data['tra'].dropna(inplace=True)
data['tes'] = data['tes'].fillna(0)

In [24]:
data['tra'] = pd.merge(data['tra'], train_r, 
                       how='left', on=['air_store_id','visit_date']).fillna(0)

In [25]:
data['tes'] = pd.merge(data['tes'], train_r, 
                       how='left', on=['air_store_id','visit_date']).fillna(0)

In [26]:
tmp_area = pd.merge(tmp_area['ar'], tmp_area['hr'], 
                    how='outer', on=['air_area_name','visit_date']).fillna(0)
tmp_area['area_reserve_visitors'] = tmp_area['area_reserve_visitors_x'] + tmp_area['area_reserve_visitors_y']

In [27]:
del tmp_area['area_reserve_visitors_x']
del tmp_area['area_reserve_visitors_y']

In [28]:
tmp_genre = pd.merge(tmp_genre['ar'], tmp_genre['hr'], 
                     how='outer', on=['air_genre_name','visit_date']).fillna(0)

In [29]:
tmp_genre['genre_reserve_visitors'] = tmp_genre['genre_reserve_visitors_x'] + tmp_genre['genre_reserve_visitors_y']
del tmp_genre['genre_reserve_visitors_x']
del tmp_genre['genre_reserve_visitors_y']

In [32]:
data['tra'] = pd.merge(data['tra'], tmp_area, 
                       how='left', on=['visit_date','air_area_name']).fillna(0)
data['tra'] = pd.merge(data['tra'], tmp_genre, 
                       how='left', on=['visit_date','air_genre_name']).fillna(0)
data['tes'] = pd.merge(data['tes'], tmp_area, 
                       how='left', on=['visit_date','air_area_name']).fillna(0)
data['tes'] = pd.merge(data['tes'], tmp_genre, 
                       how='left', on=['visit_date','air_genre_name']).fillna(0)

In [33]:
genre_mapping = {label:idx+1 for idx,label in enumerate(np.unique(data['as']['air_genre_name']))}
area_mapping = {label:idx+1 for idx,label in enumerate(np.unique(data['as']['air_area_name']))}

#!!!!categorical varibales: family, class, cluster
data['tra']['air_genre_name'] = data['tra']['air_genre_name'].map(genre_mapping)
data['tra']['air_area_name'] = data['tra']['air_area_name'].map(area_mapping)
data['tes']['air_genre_name'] = data['tes']['air_genre_name'].map(genre_mapping)
data['tes']['air_area_name'] = data['tes']['air_area_name'].map(area_mapping)

In [34]:
train_all = data['tra'].copy()
test_all = data['tes'].copy()

In [35]:
train_all.columns

Index(['air_store_id', 'visit_date', 'visitors', 'year', 'month', 'week',
       'dow', 'min_visitors', 'max_visitors', 'mean_visitors',
       'median_visitors', 'count_visitors', 'before_mean_visit',
       'before_min_visit', 'before_max_visit', 'before_median_visit',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'holiday_flg', 'reserve_visitors_x', 'reserve_visitors_y',
       'reserve_visitor', 'area_reserve_visitors', 'genre_reserve_visitors'],
      dtype='object')

In [36]:
test_all.columns

Index(['id', 'visitors', 'visit_date', 'air_store_id', 'year', 'month', 'week',
       'dow', 'min_visitors', 'max_visitors', 'mean_visitors',
       'median_visitors', 'count_visitors', 'before_mean_visit',
       'before_min_visit', 'before_max_visit', 'before_median_visit',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'holiday_flg', 'reserve_visitors_x', 'reserve_visitors_y',
       'reserve_visitor', 'area_reserve_visitors', 'genre_reserve_visitors'],
      dtype='object')

In [37]:
change_value = ['visitors','min_visitors','max_visitors',
                'mean_visitors','median_visitors','count_visitors',
                'before_mean_visit','before_min_visit','before_max_visit',
                'before_median_visit','reserve_visitor','reserve_visitors_x',
                'reserve_visitors_y','area_reserve_visitors','genre_reserve_visitors']

for name in change_value:
    train_all[name] = train_all[name].apply(lambda r:np.log1p(float(r)))
    test_all[name] = test_all[name].apply(lambda r:np.log1p(float(r)))

In [38]:
htrain = h2o.H2OFrame(train_all)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [42]:
htest = h2o.H2OFrame(test_all)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [43]:
htrain.drop(['air_store_id','visit_date','visitors'])
htest.drop(['id','visitors','visit_date','air_store_id'])
x = htrain.columns
y = 'visitors'
x.remove(y)

In [49]:
def RMSLE(y_,pred):
    return metrics.mean_squared_error(y_,pred)**0.5

In [45]:
aml = H2OAutoML(max_runtime_secs=3000)
aml.train(x=x,y=y,training_frame=htrain,leaderboard_frame=htest)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [51]:
htrain.drop(['visitors'])
preds = aml.leader.predict(htrain)
preds = preds.as_data_frame()
print('RMSLE H2O automl leader: ', RMSLE(train_all['visitors'].values, preds))

drf prediction progress: |████████████████████████████████████████████████| 100%


NameError: name 'metrics' is not defined

In [52]:
preds = aml.leader.predict(htest)
preds = preds.as_data_frame()
test_all['visitors']= preds
test_all['visitors']=np.expm1(test_all['visitors'])

drf prediction progress: |████████████████████████████████████████████████| 100%


In [53]:
test_all[['id','visitors']].to_csv('last!!.csv', index=False, float_format='%.3f')

In [54]:
test_all.shape

(32019, 27)